In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("/home/zelim/Desktop/cqa_code/cqa_quant2/data/btc_1h_candles.csv")
# Convert string to datetime
df["start_time"] = pd.to_datetime(df["start_time"])

# Convert datetime to milliseconds
df["timestamp_ms"] = df["start_time"].astype("int64") // 10**6
# window
# df = pd.read_csv(r"\home\zelim\Desktop\cqa_code\cqa_quant2\btc_1h_candles.csv")
# df

In [15]:
def zscore_backtesting(
    data_column_name, threshold, rolling_window, sr_multiplier, fees, entry_exit_type
):
    # calcuate pcg chg of close
    df["close_pct_chg"] = df["close"].pct_change()
    # use close to calculate zscore
    # calculate rolling mean of close price
    df["sma"] = df[data_column_name].rolling(window=rolling_window).mean()
    # calculate rolling std of close price
    df["std"] = df[data_column_name].rolling(window=rolling_window).std()
    # zscore = (close - mean) / std
    df["zscore"] = (df[data_column_name] - df["sma"]) / df["std"]
    # entry exit logic
    # get position
    # for loop
    zscore = df["zscore"].values
    pos = [0]
    if entry_exit_type == "trend":
        # trend follow
        # 1 = long, -1 = short , 0 = nothing
        for i in range(1, len(zscore)):
            # zscore > 1.5 = long
            if zscore[i] >= threshold:
                pos.append(1)
            # zscore < -1.5 = short
            elif zscore[i] <= -threshold:
                pos.append(-1)
            # follow previous position, hold
            else:
                pos.append(pos[i - 1])
    elif entry_exit_type == "trend_reverse":
        # 1 = short, -1 = long , 0 = nothing
        for i in range(1, len(zscore)):
            # zscore > 1.5 = short
            if zscore[i] >= threshold:
                pos.append(-1)
            # zscore < -1.5 = long
            elif zscore[i] <= -threshold:
                pos.append(1)
            # follow previous position, hold
            else:
                pos.append(pos[i - 1])
    elif entry_exit_type == "mr":
        for i in range(1, len(zscore)):
            # zscore > 1.5 = long
            if zscore[i] >= threshold:
                pos.append(1)
            # zscore < -1.5 = short
            elif zscore[i] <= -threshold:
                pos.append(-1)
            #
            elif (zscore[i] > 0.0 and pos[i - 1] == -1) or (
                zscore[i] < 0.0 and pos[i - 1] == 1
            ):
                pos.append(0)
            # follow previous position, hold
            else:
                pos.append(pos[i - 1])

    df["pos"] = pos
    # calcualte trades
    df["trades"] = abs(df["pos"] - df["pos"].shift(1))
    # calculate pnl
    df["pnl"] = df["pos"].shift(1) * df["close_pct_chg"] - df["trades"] * fees / 100
    # calculate SR, CR , Ar , MDD
    df["cumu"] = df["pnl"].cumsum()
    df["cumu_max"] = df["pnl"].cummax()
    df["drawdown"] = df["cumu"] - df["cumu_max"]
    mdd = df["drawdown"].min()
    ar = df["pnl"].mean() * 365
    sr = df["pnl"].mean() / df["pnl"].std() * np.sqrt(365 * sr_multiplier)
    cr = df["pnl"].mean() * 365 * sr_multiplier / abs(mdd)

    report = {
        "rolling_window": rolling_window,
        "threshold": threshold,
        "SR": sr,
        "CR": cr,
        "AR": ar,
        "MDD": mdd,
        "Total Return": df["cumu"].iat[-1],
        "Num of Trades": df["trades"].sum(),
    }
    return report

In [ ]:
# rolling_window from 10 to 500
# threshold from 0 to 4
rolling_window_range = np.arange(10, 500, 10)
threshold_range = np.arange(0, 4, 0.25)

all_backtest_report = []
# for loop
# 10 to 500 rolling window
for rolling_window in rolling_window_range:
    # 10
    # 20
    for threshold in threshold_range:
        backtest_report = zscore_backtesting(
            data_column_name="close",
            entry_exit_type="trend",
            sr_multiplier=24,
            fees=0.06,
            rolling_window=rolling_window,
            threshold=threshold,
        )
        all_backtest_report.append(backtest_report)

In [ ]:
report_df = pd.DataFrame(all_backtest_report)
pivot_table = report_df.pivot(index="rolling_window", columns="threshold", values="SR")
plt.figure(figsize=(30, 10))
sns.heatmap(pivot_table, annot=True, cmap="RdYlGn")
# Show the plot
plt.show()

In [ ]:
zscore_backtesting(
    data_column_name="close",
    sr_multiplier=24,
    entry_exit_type="trend",
    fees=0.06,
    rolling_window=90,
    threshold=2.75,
)

In [ ]:
df["cumu"].plot()